In [53]:
import polars as pl

In [54]:
queimadas_df = pl.read_parquet("data/queimadas-full.pqt.zstd")
municipios_df = pl.read_csv("data/municipios.csv", separator=";",encoding='iso-8859-1')
uf_df = pl.read_csv("data/uf.csv")

In [55]:
municipios_df.head()

CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
i64,i64,str,str,str
1,1100106,"""GUAJARÁ-MIRIM""","""Guajará-Mirim""","""RO"""
2,1100379,"""ALTO ALEGRE DOS PARECIS""","""Alto Alegre dos Parecis""","""RO"""
3,1100205,"""PORTO VELHO""","""Porto Velho""","""RO"""
4,1100452,"""BURITIS""","""Buritis""","""RO"""
5,1100122,"""JI-PARANÁ""","""Ji-Paraná""","""RO"""


In [56]:
uf_df.head()

id_uf,sigla,nome,regiao
i64,str,str,str
42,"""SC""","""Santa Catarina""","""Sul"""
41,"""PR""","""Paraná""","""Sul"""
43,"""RS""","""Rio Grande do Sul""","""Sul"""
11,"""RO""","""Rondônia""","""Norte"""
13,"""AM""","""Amazonas""","""Norte"""


In [57]:
queimadas_df.head()

ano,mes,data_hora,bioma,sigla_uf,id_municipio,latitude,longitude,satelite,dias_sem_chuva,precipitacao,risco_fogo,potencia_radiativa_fogo
i64,i64,datetime[μs],str,str,str,f64,f64,str,f64,f64,f64,f64
2008,1,2008-01-01 15:54:00,"""Mata Atlântica""","""BA""","""2900801""",-17.406,-39.387,null,null,null,null,null
2008,1,2008-01-01 15:55:00,"""Caatinga""","""AL""","""2702306""",-10.085,-36.342,null,null,null,null,null
2008,1,2008-01-01 15:55:00,"""Caatinga""","""SE""","""2805604""",-9.931,-37.239,null,null,null,null,null
2008,1,2008-01-01 15:55:00,"""Caatinga""","""SE""","""2807402""",-10.971,-38.002,null,null,null,null,null
2008,1,2008-01-01 15:55:00,"""Caatinga""","""SE""","""2807402""",-10.969,-37.988,null,null,null,null,null


In [58]:
# generate dim_horarios_queimada
# generate all combinations of hour and minutes in a day
minuto_list = 24*[list(range(0, 60))]
dim_horarios_full = pl.DataFrame({
    "hora": list(range(0, 24)),
    "minuto": minuto_list}).explode("minuto")
dim_horarios_full.filter((pl.col("hora") == 0))

hora,minuto
i64,i64
0,0
0,1
0,2
0,3
0,4
…,…
0,55
0,56
0,57


In [59]:
# create dim_data_queimada
dim_data_queimada = (queimadas_df
    .select([pl.col("data_hora").dt.date().alias("date_iso"),
             pl.col("data_hora").dt.day().alias("dia"), 
             pl.col("data_hora").dt.month().alias("mes"), 
             pl.col("data_hora").dt.year().alias("ano"),
             # generate semester column
             pl.col("data_hora").dt.month().map_elements(lambda x: 1 if x <= 6 else 2).alias("semestre"),
             # generate trimester column
             ((pl.col("data_hora").dt.month() - 1 )//3 + 1).alias("trimestre"),
             # generate week day column
             pl.col("data_hora").dt.weekday().alias("dia_semana"),
             # generate day of year column
             pl.col("data_hora").dt.ordinal_day().alias("dia_ano"),
             # generate is_weekend column, starting from saturday (6)
             pl.col("data_hora").dt.weekday().map_elements(lambda x: x >= 6).alias("is_weekend"),
             # generate week of year column
             pl.col("data_hora").dt.week().alias("semana_ano")
             ]).unique().sort("date_iso"))
dim_data_queimada.head()

date_iso,dia,mes,ano,semestre,trimestre,dia_semana,dia_ano,is_weekend,semana_ano
date,i8,i8,i32,i64,i8,i8,i16,bool,i8
2003-01-01,1,1,2003,1,1,3,1,false,1
2003-01-02,2,1,2003,1,1,4,2,false,1
2003-01-03,3,1,2003,1,1,5,3,false,1
2003-01-04,4,1,2003,1,1,6,4,true,1
2003-01-05,5,1,2003,1,1,7,5,true,1


In [60]:
# add season to dim_data_queimada using day and month
# AAAAAAAAAAAAAA there is no built-in season function in polars
# so we have to do it manually
# Summer - 1: Dec 21 - Mar 19
# Autumn - 2: Mar 20 - Jun 20
# Winter - 3: Jun 21 - Sep 21
# Spring - 4: Sep 22 - Dec 20
dim_data_queimada = dim_data_queimada.with_columns(
    pl.when(( (pl.col("mes") == 12) & (pl.col("dia") >= 21) ) | (pl.col("mes").is_in([1,2])) | ((pl.col("mes") == 3) & (pl.col("dia") < 20)))
    .then(1)
    .when(( (pl.col("mes") == 3) & (pl.col("dia") >= 20) ) | (pl.col("mes").is_in([4,5])) | ((pl.col("mes") == 6) & (pl.col("dia") < 21)))
    .then(2)
    .when(( (pl.col("mes") == 6) & (pl.col("dia") >= 21) ) | (pl.col("mes").is_in([7,8])) | ((pl.col("mes") == 9) & (pl.col("dia") < 22)))
    .then(3)
    .otherwise(4).alias("estacao")
)
dim_data_queimada.head()

date_iso,dia,mes,ano,semestre,trimestre,dia_semana,dia_ano,is_weekend,semana_ano,estacao
date,i8,i8,i32,i64,i8,i8,i16,bool,i8,i32
2003-01-01,1,1,2003,1,1,3,1,false,1,1
2003-01-02,2,1,2003,1,1,4,2,false,1,1
2003-01-03,3,1,2003,1,1,5,3,false,1,1
2003-01-04,4,1,2003,1,1,6,4,true,1,1
2003-01-05,5,1,2003,1,1,7,5,true,1,1


In [61]:
# generate dim_local_queimada
dim_local_queimada = (queimadas_df
    .select([pl.col("id_municipio").alias("id_municipio").cast(pl.Int32),
             pl.col("sigla_uf").alias("sigla_uf"),
             pl.col("bioma").alias("bioma"),
             pl.col("latitude").alias("latitude"),
             pl.col("longitude").alias("longitude"),]).unique().sort(["id_municipio", "sigla_uf"]))
dim_local_queimada.head()

id_municipio,sigla_uf,bioma,latitude,longitude
i32,str,str,f64,f64
1100015,"""RO""","""Amazônia""",-12.194,-62.165
1100015,"""RO""","""Amazônia""",-12.882,-62.206
1100015,"""RO""","""Amazônia""",-12.93237,-62.19844
1100015,"""RO""","""Amazônia""",-12.93405,-62.4535
1100015,"""RO""","""Amazônia""",-12.965,-62.155


In [62]:
# join with municipios to get municipio name
# join with uf to get uf name and regiao
dim_local_queimada = (dim_local_queimada
    .join(municipios_df.select([pl.col("MUNICÍPIO - IBGE"), pl.col('CÓDIGO DO MUNICÍPIO - IBGE')]), left_on="id_municipio", right_on="CÓDIGO DO MUNICÍPIO - IBGE", how="left")
    .join(uf_df.select([pl.col("sigla"), pl.col("nome"), pl.col("regiao")]), left_on="sigla_uf", right_on="sigla", how="left"))
dim_local_queimada = dim_local_queimada.select([
    pl.col("id_municipio"),
    pl.col("MUNICÍPIO - IBGE").alias("nome_municipio"),
    pl.col("sigla_uf"),
    pl.col("nome").alias("nome_uf"),
    pl.col("regiao").alias("regiao_uf"),
    pl.col("bioma"),
    pl.col("latitude"),
    pl.col("longitude"),
])
dim_local_queimada.head()

id_municipio,nome_municipio,sigla_uf,nome_uf,regiao_uf,bioma,latitude,longitude
i32,str,str,str,str,str,f64,f64
1100015,"""Alta Floresta D'Oeste""","""RO""","""Rondônia""","""Norte""","""Amazônia""",-12.194,-62.165
1100015,"""Alta Floresta D'Oeste""","""RO""","""Rondônia""","""Norte""","""Amazônia""",-12.882,-62.206
1100015,"""Alta Floresta D'Oeste""","""RO""","""Rondônia""","""Norte""","""Amazônia""",-12.93237,-62.19844
1100015,"""Alta Floresta D'Oeste""","""RO""","""Rondônia""","""Norte""","""Amazônia""",-12.93405,-62.4535
1100015,"""Alta Floresta D'Oeste""","""RO""","""Rondônia""","""Norte""","""Amazônia""",-12.965,-62.155
